In [34]:
# importing the tools required for the Poisson regression model
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
from scipy.stats import poisson,skellam
import prettytable as pt

In [35]:
# SET TEAMS HERE
home_team='Arsenal'
away_team='Manchester City'

In [47]:
epl = pd.read_csv(r"C:\Users\blake\OneDrive\Documents\Pl_2223 xG.csv",encoding='latin1')
ep = epl[['Home','Away','xGh','xGa']]
epl = epl.rename(columns={'xGh': 'HomeGoals', 'xGa': 'AwayGoals'})

In [48]:
goal_model_data = pd.concat([epl[['Home','Away','HomeGoals']].assign(home=1).rename(
            columns={'Home':'team', 'Away':'opponent','HomeGoals':'goals'}),
           epl[['Away','Home','AwayGoals']].assign(home=0).rename(
            columns={'Away':'team', 'Home':'opponent','AwayGoals':'goals'})])

poisson_model = smf.glm(formula="goals ~ home + team + opponent", data=goal_model_data,
                        family=sm.families.Poisson()).fit()
poisson_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  goals   No. Observations:                  758
Model:                            GLM   Df Residuals:                      718
Model Family:                 Poisson   Df Model:                           39
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -937.23
Date:                Fri, 04 Aug 2023   Deviance:                       228.42
Time:                        21:44:06   Pearson chi2:                     228.
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1346
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                       0.2771      0.200      1.388      0.165      -0.114       0.668
team[T.Aston Villa]            -0.3774      0.187     -2.013      0.044      -0.745      -0.010
team[T.Bournemouth]            -0.6077      0.200     -3.032      0.002      -1.000      -0.215
team[T.Brentford]              -0.2221      0.178     -1.250      0.211      -0.570       0.126
team[T.Brighton]                0.0319      0.167      0.191      0.849      -0.296       0.360
team[T.Chelsea]                -0.3594      0.185     -1.944      0.052      -0.722       0.003
team[T.Crystal Palace]         -0.5968      0.199     -3.004      0.003      -0.986      -0.207
team[T.Everton]                -0.4374      0.190     -2.301      0.021      -0.810      -0.065
team[T.Fulham]                 -0.4175      0.189     -2.211      0.027      -0.788      -0.047
team[T.Leeds United]           -0.3885      0.187     -2.073      0.038      -0.756      -0.021
team[T.Leicester City]         -0.3307      0.184     -1.797      0.072      -0.691       0.030
team[T.Liverpool]               0.0198      0.167      0.118      0.906      -0.307       0.347
team[T.Manchester City]         0.0836      0.163      0.511      0.609      -0.237       0.404
team[T.Manchester Utd]         -0.0526      0.170     -0.309      0.757      -0.385       0.280
team[T.Newcastle Utd]           0.0033      0.167      0.020      0.984      -0.324       0.331
team[T.Nott'ham Forest]        -0.5763      0.198     -2.904      0.004      -0.965      -0.187
team[T.Southampton]            -0.6206      0.201     -3.087      0.002      -1.015      -0.227
team[T.Tottenham]              -0.2177      0.177     -1.227      0.220      -0.565       0.130
team[T.West Ham]               -0.3670      0.185     -1.980      0.048      -0.730      -0.004
team[T.Wolves]                 -0.6428      0.202     -3.175      0.002      -1.040      -0.246
opponent[T.Aston Villa]         0.2015      0.208      0.967      0.333      -0.207       0.610
opponent[T.Bournemouth]         0.3802      0.199      1.913      0.056      -0.009       0.770
opponent[T.Brentford]           0.1616      0.209      0.773      0.439      -0.248       0.571
opponent[T.Brighton]            0.1394      0.212      0.657      0.511      -0.277       0.556
opponent[T.Chelsea]             0.1973      0.207      0.953      0.340      -0.208       0.603
opponent[T.Crystal Palace]      0.1018      0.211      0.482      0.630      -0.312       0.515
opponent[T.Everton]             0.4180      0.198      2.116      0.034       0.031       0.805
opponent[T.Fulham]              0.3880      0.199      1.952      0.051      -0.002       0.778
opponent[T.Leeds United]        0.4413      0.197      2.243      0.025       0.056       0.827
opponent[T.

In [46]:
#Predict for Home Team vs. Away Team
home_score_rate=poisson_model.predict(pd.DataFrame(data={'team': home_team, 'opponent': away_team,
                                       'home':1},index=[1]))
away_score_rate=poisson_model.predict(pd.DataFrame(data={'team': away_team, 'opponent': home_team,
                                       'home':0},index=[1]))
print(home_team + ' against ' + away_team + ' xG: ' + str(home_score_rate))
print(away_team + ' against ' + home_team + ' xG: ' + str(away_score_rate))

#Lets just get a result
home_goals=np.random.poisson(home_score_rate)
away_goals=np.random.poisson(away_score_rate)

Arsenal against Manchester City xG: 1    1.269594
dtype: float64
Manchester City against Arsenal xG: 1    1.434234
dtype: float64


In [40]:
# Code to caluclate the goals for the match.
def simulate_match(foot_model, homeTeam, awayTeam, max_goals=10):
    home_goals_avg = foot_model.predict(pd.DataFrame(data={'team': homeTeam,
                                                           'opponent': awayTeam, 'home': 1},
                                                     index=[1])).values[0]
    away_goals_avg = foot_model.predict(pd.DataFrame(data={'team': awayTeam,
                                                           'opponent': homeTeam, 'home': 0},
                                                     index=[1])).values[0]
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals + 1)] for team_avg in
                 [home_goals_avg, away_goals_avg]]
    return (np.outer(np.array(team_pred[0]), np.array(team_pred[1])))

#Matrix
max_goals=5
score_matrix=simulate_match(poisson_model, home_team, away_team,max_goals)

#Home, draw, away probabilities
homewin=np.sum(np.tril(score_matrix, -1))
draw=np.sum(np.diag(score_matrix))
awaywin=np.sum(np.triu(score_matrix, 1))

In [45]:
table = pt.PrettyTable()

table.field_names = ['Team', 'Goals', 'Win %']

table.add_row([home_team, home_goals[0], round(homewin*100, 2)])
table.add_row([away_team, away_goals[0], round(awaywin*100, 2)])

print(table.get_string())

+-----------------+-------+-------+
|       Team      | Goals | Win % |
+-----------------+-------+-------+
|     Arsenal     |   0   | 33.17 |
| Manchester City |   2   | 40.58 |
+-----------------+-------+-------+
